# Multi-Output model example: A social media model with three heads
# ![](../materials/social_media_model.png)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(inputs=posts_input, 
              outputs=[age_prediction, income_prediction, gender_prediction])

# Compilation options of a multi-output model: multiple losses
# 1
model.compile(  optimizer='rmsprop', 
                loss=[  'mse', 
                        'categorical_crossentropy', 
                        'binary_crossentropy'])

# 2 (Equivalent to #1)
# Possible only if you give names to the output layers
model.compile(  optimizer='rmsprop', 
                loss={  'age': 'mse', 
                        'income': 'categorical_crossentropy', 
                        'gender': 'binary_crossentropy'})

# 3
# Assign different levels of importance to the loss values in their contribution to the final loss
model.compile(  optimizer='rmsprop', 
                loss=[  'mse', 
                        'categorical_crossentropy', 
                        'binary_crossentropy'], 
                loss_weights=[0.25, 1., 10.])

# 4 (Equivalent to #3)
model.compile(  optimizer='rmsprop', 
                loss={  'age': 'mse', 
                        'income': 'categorical_crossentropy', 
                        'gender': 'binary_crossentropy'}, 
                loss_weights={  'age': 0.25, 
                                'income': 1., 
                                'gender': 10.})

# Feeding data to a multi-output model
# 1
model.fit(  posts, 
            [age_targets, income_targets, gender_targets], 
            epochs=10, 
            batch_size=64)

# 2
model.fit(  posts, 
            {'age': age_targets,
             'income': income_targets,
             'gender': gender_targets},
            epochs=10,
            batch_size=64)
